# 06. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
# YOUR CODE HERE
%%writefile get_data.py
"""
File downloads, unzips, and sets up the model directories used for training a image classification model
"""
import os
import zipfile
from pathlib import Path
import requests

data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

# Check/ Create data folder
if image_path.is_dir():
  print(f'Directory {image_path} exists already')
else:
  print(f'Making directory {image_path}....')
  image_path.mkdir(exist_ok=True, parents=True)

# Download and unzip
with open(str(image_path)+'.zip', 'wb' ) as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print(f'Downloading zip file at {str(image_path)+'.zip'}')
  print()
  f.write(request.content)

with zipfile.ZipFile(str(image_path)+'.zip', 'r') as zip_ref:
  print(f'Unzipping image folder {str(image_path)+'.zip'}')
  zip_ref.extractall(image_path)

os.remove(str(image_path)+'.zip')

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py

Making directory data/pizza_steak_sushi....

Unzipping image folder data/pizza_steak_sushi.zip


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [3]:
# Create a directory
import os
os.makedirs('going_modular', exist_ok=True)

In [4]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for image classification data.
"""
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

num_workers = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = num_workers
):
  """
  Creates training and testing DataLoaders.
  Takes in a training and testing directory path and turns them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader

  Returns:
  A tuple of (train_dataloader, test_dataloader, class_names).
  Where class_names is a list of the target classes.

  Example usage:
    train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir, test_dir=path/to/test_dir, transform=some_transform, batch_size=32, num_workers=4)
  """

  print('Creating Datasets........')
  train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
  test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
  print(f'Train and Test datasets created')

  print('Creating DataLoaders........')
  train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
  test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True)
  print(f'Train and Test DataLoaders created')

  class_names = train_dataset.classes

  return train_dataloader, test_dataloader, class_names


Writing going_modular/data_setup.py


In [5]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model,
from the CNN explainer website
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates a TinyVGG architecture.

  Replicates the TinyVGG acrchitecture from the CNN explainer website in Pytorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels,
    hidden_units: An integer indicating number of hidden units between layers,
    output_shape: An integer indicating number of outptu units

  """
  def __init__(self, input_shape: int, hidden_units:int, output_shape:int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # input_featuer len comes form hidden units being compressed
        nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x



Writing going_modular/model_builder.py


In [6]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing PyTorch model.
"""
from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device) -> Tuple[float, float]:
  """ Trains a PyTorch model for a single epoch.

  Truns a target PyTorch model to training mode and then runs through all the required
  training steps (forward pass, loss calculation, optimizer step).

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorcch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.train()

  train_loss = train_acc = 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  train_loss = train_loss/ len(dataloader)
  train_acc = train_acc/ len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, device: torch.device) -> Tuple[float, float]:
  """ Tests a PyTorch model for a single epoch.

  Truns a target PyTorch model to eval mode and then runs forward pass.

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.eval()

  test_loss = test_acc = 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

from typing import Dict, List
from tqdm.auto import tqdm

def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device, epochs: int) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []
             }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model, dataloader=train_dataloader, loss_fn=loss_fn, device=device, optimizer=optimizer)
    test_loss, test_acc = test_step(model=model, loss_fn=loss_fn, dataloader=test_dataloader, device=device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)
    results['train_acc'].append(train_acc)
    results['test_acc'].append(test_acc)

  return results

Writing going_modular/engine.py


In [7]:
%%writefile going_modular/utils.py
"""
Contains function to save the PyTorch model
"""
import torch
from pathlib import Path

def save_model(model:torch.nn.Module, target_dir:str, model_name:str):
  """ Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should Include either ".pth' or ".pt"

  Example usage:
    save_model(model=model_0,
                target_dir="models",
                model_name="05_going_modular_tingvgg_model.pth")
  """
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(exist_ok=True, parents=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith('.pt'), 'model_name should end with .pth or .pt'
  model_save_path = target_dir_path/model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

Writing going_modular/utils.py


In [8]:
# YOUR CODE HERE
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.

Args:
    train: Training directory
    test: Testing directory
    learning_rate: Learning rate
    batch_size: Batch size
    num_epochs: Number of epochs to train for
    hidden_units: Number of hidden units in the TinyVGG model
"""
import torch
import os
from torchvision import transforms
import data_setup
import utils
import engine
import model_builder
from timeit import default_timer as timer
import argparse

parser = argparse.ArgumentParser()

# Set seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Setup Hyper parameters
learning_rate = 0.001
num_epochs = 3
batch_size = 32
hidden_units = 10
input_shape = 3


# Setup directories
train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'

parser.add_argument('-tr', '--train', default=train_dir, help='Training directory')
parser.add_argument('-te', '--test', default=test_dir, help='Testing directory')
parser.add_argument('-lr', '--learning_rate', type=float, default=learning_rate, help='Learning rate')
parser.add_argument('-b', '--batch_size', type=int, default=batch_size, help='Batch size')
parser.add_argument('-e', '--num_epochs', type=int, default=num_epochs, help='Number of epochs to train for')
parser.add_argument('-hi', '--hidden_units', type=int, default=hidden_units, help='Number of hidden units in the TinyVGG model')

args = parser.parse_args()

train_dir = args.train
test_dir = args.test
learning_rate = args.learning_rate
num_epochs = args.num_epochs
batch_size = args.batch_size
hidden_units = args.hidden_units

# Device agnostice code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transformmation
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

# Create dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            transform=data_transform,
                                                            batch_size=batch_size)

# Create model
model = model_builder.TinyVGG(input_shape=input_shape, hidden_units=hidden_units, output_shape=len(class_names))

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

start_time = timer()

# Train model with engine
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device,
             epochs=num_epochs
             )

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model using utils
utils.save_model(model=model,
                 model_name='06_going_modular_script_mode.pth',
                 target_dir='models')

Writing going_modular/train.py


In [9]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

Creating Datasets........
Train and Test datasets created
Creating DataLoaders........
Train and Test DataLoaders created
  0% 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch: 1 | train_loss: 1.1031 | train_acc: 0.3137 | test_loss: 1.0937 | test_acc: 0.3333
 20% 1/5 [00:30<02:02, 30.59s/it]Epoch: 2 | train_loss: 1.0942 | train_acc: 0.3434 | test_loss: 1.0972 | test_acc: 0.3467
 40% 2/5 [01:05<01:38, 32.99s/it]Epoch: 3 | train_loss: 1.0791 | train_acc: 0.4221 | test_loss: 1.0757 | test_acc: 0.4000
 60% 3/5 [01:41<01:08, 34.48s/it]Epoch: 4 | train_loss: 1.0478 | train_acc: 0.5049 | test_loss: 1.0443 | test_acc: 0.4667
 80% 4/5 [02:13<00:33, 33.39s/it]Epoch: 5 | train_loss: 0.9915 | train_acc: 0.5310 | test_loss: 1.0381 | test_acc: 0.4533
100% 5/5 [02:43<00:00, 32.79s/it]
[INFO] Total trainin

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [17]:
# YOUR CODE HERE
%%writefile going_modular/predict.py
"""
Contains code which loads pre-saved model and predicts on custom image
"""
import torch
import torchvision
from torchvision import transforms
import argparse
from pathlib import Path
from typing import List
import model_builder

# Setup argument parser
parser = argparse.ArgumentParser(description='Predict on custom image using trained model')

parser.add_argument('-img', '--image', type=str, required=True, help='Path to image for prediction')
parser.add_argument('-m', '--model_path', type=str, default='models/06_going_modular_script_mode.pth', help='Path to trained model')
parser.add_argument('-d', '--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu',
                    choices=['cuda', 'cpu'], help='Device to run inference on')
parser.add_argument('-hu', '--hidden_units', type=int, default=10, help='Number of hidden units in model')
parser.add_argument('-in', '--input_shape', type=int, default=3, help='Number of input channels')

args = parser.parse_args()

# Setup device
device = args.device

# Setup transformation
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
])

def pred_and_plot_image(
    model: torch.nn.Module,
    image_path: str,
    class_names: List[str] = None,
    transform=None,
    device: torch.device = device
):
    """Makes a prediction for a given custom image once its path is known"""
    # Load in the image and scale it
    target_image = torchvision.io.read_image(str(image_path)).type(torch.float32) / 255

    # Transform image
    if transform is not None:
        target_image_transformed = transform(target_image)
    else:
        target_image_transformed = target_image

    # Add batch dimension
    target = target_image_transformed.unsqueeze(0)

    # Make sure both are on same device
    target = target.to(device)
    model.to(device)

    # Pass image through the model
    model.eval()
    with torch.inference_mode():
        target_logits = model(target)
        target_preds = target_logits.softmax(1).argmax(1)

    # Display class name
    if class_names is not None:
        print(f"Image belongs to: {class_names[target_preds]}")
    else:
        print(f"Predicted class: {target_preds}")

# Load the saved model state
class_names = ['pizza', 'steak', 'sushi']
saved_state = torch.load(args.model_path, map_location=device)

# Try to infer hidden_units from the saved model if not specified
if 'conv_block_1.0.weight' in saved_state:
    inferred_hidden_units = saved_state['conv_block_1.0.weight'].shape[0]
    print(f"[INFO] Detected hidden_units={inferred_hidden_units} from saved model")
    hidden_units = inferred_hidden_units
else:
    hidden_units = args.hidden_units
    print(f"[INFO] Using hidden_units={hidden_units} from arguments")

# Create model with correct architecture
model = model_builder.TinyVGG(
    input_shape=args.input_shape,
    hidden_units=hidden_units,
    output_shape=len(class_names)
)

# Load saved weights
model.load_state_dict(saved_state)
print(f"[INFO] Model loaded successfully from {args.model_path}")

# Make prediction
pred_and_plot_image(
    model=model,
    image_path=args.image,
    class_names=class_names,
    transform=data_transform,
    device=device
)

Overwriting going_modular/predict.py


In [18]:
# Example running of predict.py
!python going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Detected hidden_units=128 from saved model
[INFO] Model loaded successfully from models/06_going_modular_script_mode.pth
Image belongs to: sushi
